In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
seed = 1
df = pd.read_excel('data/raw/new_clean_data_all.xlsx')
print(len(df))
print(df.columns)

46588
Index(['Unnamed: 0', 'No.', 'TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱',
       '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為', '其他類型'],
      dtype='object')


In [3]:
other_type = []
for idx, row in df.iterrows():
    if row['無助或無望'] or row['其他負向文字'] or row['生理反應或醫療狀況']:
        other_type.append(1)
    else:
        other_type.append(0)
        if row['正向文字']:
            df['無標註'][idx] = 1
df['其他類型'] = other_type

/tmp/ipykernel_2565347/2306923281.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['無標註'][idx] = 1


In [4]:
# df.to_excel("data/final/sentence/four_class_0530.xlsx")

In [5]:
df = pd.read_excel('data/final/sentence/four_class_0530.xlsx')
target_columns = ['TextID' , 'Title' ,'Sentence','無標註','自殺與憂鬱','自殺行為','其他類型']
df = df[target_columns]
all_labelize_df = []
new_column=['無標註','自殺與憂鬱','自殺行為','其他類型']
for label in new_column:
    all_labelize_df.append(df[df[label] == 1].loc[:])
for i in all_labelize_df:
    print(len(i))
    print(i.columns)

34808
Index(['TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為', '其他類型'], dtype='object')
3443
Index(['TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為', '其他類型'], dtype='object')
139
Index(['TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為', '其他類型'], dtype='object')
8198
Index(['TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為', '其他類型'], dtype='object')


In [6]:
all_train_df = [[] for _ in range(5)]
all_test_df = [[] for _ in range(5)]
skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
for i in range(len(all_labelize_df)):
    for idx, (train_index, test_index) in enumerate(skf.split(all_labelize_df[i], [0]*len(all_labelize_df[i]))):
        train_df = all_labelize_df[i].iloc[train_index]
        test_df = all_labelize_df[i].iloc[test_index]
        all_train_df[idx].append(train_df)
        all_test_df[idx].append(test_df)
    
for i in range(5):
    print("Fold", i)
    train_df = pd.concat(all_train_df[i], axis=0)
    test_df = pd.concat(all_test_df[i], axis=0)
    print(train_df.columns)
    # show number of each class
    for label in new_column:
        print("Label:", label)
        print("number of train data:", len(train_df[train_df[label] == 1]))
        print("number of test data:", len(test_df[test_df[label] == 1]))
    # save to excel
    # train_df.to_excel("data/final/sentence/four_class_0530_train_fold_{}.xlsx".format(i))
    # test_df.to_excel("data/final/sentence/four_class_0530_test_fold_{}.xlsx".format(i))
    


Fold 0
Index(['TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為', '其他類型'], dtype='object')
Label: 無標註
number of train data: 27846
number of test data: 6962
Label: 自殺與憂鬱
number of train data: 2754
number of test data: 689
Label: 自殺行為
number of train data: 111
number of test data: 28
Label: 其他類型
number of train data: 6558
number of test data: 1640
Fold 1
Index(['TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為', '其他類型'], dtype='object')
Label: 無標註
number of train data: 27846
number of test data: 6962
Label: 自殺與憂鬱
number of train data: 2754
number of test data: 689
Label: 自殺行為
number of train data: 111
number of test data: 28
Label: 其他類型
number of train data: 6558
number of test data: 1640
Fold 2
Index(['TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為', '其他類型'], dtype='object')
Label: 無標註
number of train data: 27846
number of test data: 6962
Label: 自殺與憂鬱
number of train data: 2754
number of test data: 689
Label: 自殺行為
number of train data: 111
number of test data: 28
Label: 其他類型


In [7]:
df = pd.read_excel('data/final/sentence/four_class/seed_1/four_class_0530_train_fold_1.xlsx')
target_columns = ['TextID' , 'Title' ,'Sentence','無標註','自殺與憂鬱','自殺行為','其他類型']
df = df[target_columns]
labelize_df = []
new_column=['無標註','自殺與憂鬱','自殺行為','其他類型']
for label in new_column:
    labelize_df.append(df[df[label] == 1].loc[:])
for i in labelize_df:
    print(len(i))
    # print(i.columns)

27846
2754
111
6558


In [9]:
# split Augmentation Data
all_middle_df = labelize_df[0]
print(all_middle_df.shape)
to_add = [4000, 6000]
Augment_df = []
for i in range(2):
    Augment_df.append(all_middle_df.sample(n=to_add[i], random_state=seed))
    all_middle_df = all_middle_df.drop(Augment_df[-1].index)
print(all_middle_df.shape)
labelize_df[0] = all_middle_df

for i in range(len(labelize_df)):
    labelize_df[i] = labelize_df[i].reset_index(drop=True)
# for df in labelize_df:
#     print(df)
for i in range(len(Augment_df)):
    Augment_df[i] = Augment_df[i].reset_index(drop=True)
for df in Augment_df:
    print(df.shape)

(17846, 7)
(7846, 7)
(4000, 7)
(6000, 7)


In [11]:
### new block at 11/13!!! Goal: take A2 sentence into training data
df_A2 = pd.read_excel('data/article/split_output_A2_v2_answer.xlsx')
df_A2 = df_A2[target_columns]
# DF = pd.concat([df, df_A2], axis=0, ignore_index=True)
all_labelize_df_A2 = []
for label in new_column:
    all_labelize_df_A2.append(df_A2[df_A2[label] == 1].loc[:])
for i in all_labelize_df_A2:
    print(len(i))

29460
4431
133
9844


In [12]:
# ### new block at 11/13!!! Goal: combine A1 training data and A2 training data
for i in range(len(labelize_df)):
    labelize_df[i] = pd.concat([labelize_df[i], all_labelize_df_A2[i]], axis=0, ignore_index=True)
    print(labelize_df[i].shape)

(37306, 7)
(7185, 7)
(244, 7)
(16402, 7)


In [13]:
print(len(labelize_df))

4


In [14]:

to_train = [16000, 0, 0, 16000]
target_num = 16000
training_df = []

augment_id = 0
for idx, a_df in enumerate(labelize_df):
    print("labelize: ",a_df.shape)
    # augment_label = new_column[idx]
    if a_df.shape[0] > target_num:
        training_df.append(a_df.sample(n=to_train[idx], random_state=seed))
        print("augment: ",training_df[-1].shape)
    else:
        to_augment_df = a_df
        boundary = a_df.shape[0]
        for i in range(len(Augment_df[augment_id])):
            # try: 
                # print(idx, i)
                if len( str(Augment_df[augment_id].at[i, 'Sentence'])) < 5:
                    augment_sentence = str(a_df.at[i % boundary, 'Sentence']) + str(Augment_df[augment_id].at[i, 'Sentence'])
                else:
                    augment_sentence = str(a_df.at[i % boundary, 'Sentence']) + str(Augment_df[augment_id].at[i, 'Sentence'])[:5]
                to_augment_df = pd.concat([to_augment_df, a_df.iloc[i % boundary:(i % boundary)+1]], ignore_index=True)
                to_augment_df.at[boundary+i, 'Sentence'] = augment_sentence
            # except:
            #     print(i % boundary)
        training_df.append(to_augment_df)
        print("augment: ",to_augment_df.shape)
        augment_id += 1
        to_add = to_train
for df in training_df:
    print(df.shape)

labelize:  (37306, 7)
augment:  (16000, 7)
labelize:  (7185, 7)
augment:  (11185, 7)
labelize:  (244, 7)
augment:  (6244, 7)
labelize:  (16402, 7)
augment:  (16000, 7)
(16000, 7)
(11185, 7)
(6244, 7)
(16000, 7)


In [67]:
augmented_df = pd.concat(training_df,axis=0, ignore_index=True)
augmented_df.to_excel('data/final/sentence/four_class/seed_1/four_class_0530_train_augmented_fold_1.xlsx')